In [1]:
from sqlalchemy import create_engine
import pandas as pd
import yaml
import numpy as np
import sklearn

In [2]:
def connecting_to_RDS():
    #with open(creds, 'r') as f:
        #creds = yaml.safe_load(f)
    DATABASE_TYPE = 'postgresql'
    DBAPI = 'psycopg2'
    ENDPOINT = 'products.c8k7he1p0ynz.us-east-1.rds.amazonaws.com'
    DBUSER = 'postgres'
    DBPASSWORD = 'aicore2022!'
    PORT = 5432
    DATABASE = 'postgres'
    engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{DBUSER}:"
                                f"{DBPASSWORD}@{ENDPOINT}:"
                                    f"{PORT}/{DATABASE}")
    engine.connect()
    main_df = pd.read_sql_table(
            'products', engine,
        columns=["id", "product_name", "category", "product_description",
                     "price", "location",
                     "page_id", "create_time"])
    return main_df
fb_df = connecting_to_RDS()

In [ ]:
fb_df.shape
#8091 rows and 8 columns

In [3]:
def remove_n_a_records(df, column: str):
    """
    Scan the column for records with all N/As. Get rid of them

    Args:
        column (str): The column currently being scanned.
    """
    # Swap N/A for the pandas nan, so we can drop them
    temp_df = df[column].replace('N/A', np.nan)
    temp_df = temp_df.dropna()
    # Create a new df with only the records without the nans
    clean_df = pd.merge(temp_df, df,
                        left_index=True, right_index=True)
    # The merge creates a duplicate column. Remove it.
    clean_df.drop(column + '_x', inplace=True, axis=1)
    # Rename the remaining category column
    clean_df.rename(columns={column + '_y': column}, inplace=True)
    # Commit the cleansed data to the dataframe
    df = clean_df
    return df

fb_df = remove_n_a_records(fb_df, 'category')

In [ ]:
fb_df

In [ ]:
#seperate numerical columns and select numerical columns
df_numeric = fb_df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

In [ ]:
#select non-numeric columns
df_non_numeric = fb_df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

In [4]:
fb_df['price'] = fb_df['price'].str.strip('£')
fb_df['price'] = fb_df['price'].str.replace(',','')
fb_df['price'] = fb_df['price'].astype('float64')

In [ ]:
fb_df['price'].head(10)

In [ ]:
#check for outliers
fb_df['price'].describe()

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [5]:
def remove_price_outliers(df):
    
    df = df[df['price'] < 1000]
    df = df[df['price'] > 1]
    return df

fb_df = remove_price_outliers(fb_df)

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [ ]:
#check for outliers
fb_df['price'].describe()

In [ ]:
#fixing data type
fb_df['create_time'].dtype

In [6]:
#convert create_time to datetime format
fb_df['create_time'] = pd.to_datetime(fb_df.create_time, format='%Y-%m-%d')

In [ ]:
fb_df.info()

In [ ]:
fb_df.duplicated()

In [ ]:
fb_df.duplicated().sum()

In [ ]:
#check for duplicated values
# dropping duplicates by considering all columns other than ID
cols_other_than_id = list(fb_df.columns)[1:]
fb_df.drop_duplicates(subset=cols_other_than_id, inplace=True)
fb_df.duplicated()

In [7]:
new_columns = ['product_name', 'product_description', 'location']
fb_df.drop_duplicates(subset=new_columns, keep='first')
fb_df[new_columns]

,product_name,product_description,location
1,"Mirror wall art | in Wokingham, Berkshire | Gu...","Mirror wall art. Posted by Nisha in Dining, Li...","Wokingham, Berkshire"
2,"Stainless Steel Food Steamer | in Inverness, H...",Morphy Richard’s (model no 48755)Stainless ste...,"Inverness, Highland"
3,"Sun loungers | in Skegness, Lincolnshire | Gum...",I have 2 of these - collection only as I don’t...,"Skegness, Lincolnshire"
4,Coffee side table from Ammunition ammo box hai...,Great reclaimed army ammunition box used as co...,"Radstock, Somerset"
5,Modern Shannon Sofa for sale at low cost | in ...,New Design Shannon Corner sofa 5 Seater Avail...,"Delph, Manchester"
...,...,...,...
8085,Sony PlayStation VR Move Bundle | in Acocks Gr...,Sony PlayStation VR Move Bundle353CASH ON COLL...,"Acocks Green, West Midlands"
8086,"Playstation VR V2 Bundle | in Acocks Green, We...",Playstation VR V2 Bundle355CASH ON COLLECTION ...,"Acocks Green, West Midlands"
8088,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Pick up only £250Comes with two pistols stocks...,"Montrose, Angus"
8089,Logitech driving force shifter | in Carrickfer...,Bought at christmas from currys retailing at £...,"Carrickfergus, County Antrim"


In [8]:
#create new dataset
X = fb_df.drop(columns=['price'])
y = fb_df['price']

In [9]:
import seaborn as sns

In [ ]:
sns.boxplot(x='category', y='price', data=fb_df)

In [13]:
# Create the model
product_X = fb_df[['product_name']]
product_y = fb_df['price']

product_X = pd.get_dummies(product_X, drop_first=True)

product_X

,"product_name_'Game Of Life Zapped' Board Game (new) | in Reading, Berkshire | Gumtree","product_name_'Magic The Gathering' Starter Game & Video (1999) | in Reading, Berkshire | Gumtree","product_name_'Salem' by Philip Martin - LIMITED Edition print | in Toton, Nottinghamshire | Gumtree","product_name_'The Famous Robin Hood' Card Game | in Reading, Berkshire | Gumtree","product_name_(2018) Top Range MacBook Pro 15” TouchBar| 4.3GHz 6 Core i7 | 16GB Memory | 512GB SSD | in Ealing Broadway, London | Gumtree","product_name_(IPH-248) iPhone 6 16GB Vodafone | in Worcester Park, London | Gumtree","product_name_(NEW) BEIGE BOOTS (SIZE 6 ) | in Swindon, Wiltshire | Gumtree","product_name_(NEW) BLACK & GOLD SANDALS (SIZE 4) | in Swindon, Wiltshire | Gumtree","product_name_(NEW) GEORGE WEDGE HEEL SANDALS (SIZE 6) | in Swindon, Wiltshire | Gumtree","product_name_(NEW) SAINSBURY’S BLACK SUEDE BOOTS (SIZE 5) | in Swindon, Wiltshire | Gumtree",...,"product_name_💕JG Melts And More💕 | in Dumfries, Dumfries and Galloway | Gumtree","product_name_💥DIVAN STORAGE BED💥HEADEBOARD|BASE||SINGLE||DOUBLE||KING||SMALL DOUBLE||MATTRESS OPT|CALL NOW📞 | in East London, London | Gumtree","product_name_💥💥2 Seater Sofa💥💥 Brand New L shape Corner Sofa in 3 seater and 2 seater | in Harrogate, North Yorkshire | Gumtree","product_name_💥💥White Wardrobe💥💥Brand new Wardrobe For sale in Black/White/Grey Colour **Mirrored Wardrobe** | in Huddersfield, West Yorkshire | Gumtree","product_name_🔥Brand New Wardrobe Sets, Top Quality and Fully Assembled | in Liverpool City Centre, Merseyside | Gumtree","product_name_🔥🔥🔥log burners🔥🔥🔥🔥🔥🔥🔥 | in Kelty, Fife | Gumtree","product_name_🚒 FAST POSTAGE ! Men Women Boys Girls Tracksuit Hoodie Bottom Joggers Jogging X,S,M,L,XL not Nike | in East Croydon, London | Gumtree","product_name_🛎 TOWER T24021 DIGITAL SOLO MICROWAVE - BLACK AND ROSE GOLD | in Haringey, London | Gumtree","product_name_🟩 1 X Perching Chair & 1 X Stool | in Atherton, Manchester | Gumtree","product_name_🟩 Folding Wheelchair Only has one Footrest/Hanger | in Atherton, Manchester | Gumtree"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

product_X_train, product_X_test, product_y_train, product_y_test = train_test_split(product_X, product_y, test_size=0.2, random_state=0)

regr = LinearRegression()
regr.fit(product_X_train, product_y_train)

product_y_pred = regr.predict(product_X_test)
product_y_pred



array([-3.17017091e+15,  9.89589190e+15,  3.00000000e+01, ...,
        3.00000000e+01,  3.00000000e+01, -1.77713060e+14])